In [2]:
import os
import requests

run code cell below to retrieve updated ONI data

In [3]:
url = "https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt"
path = "data/oni.ascii.txt"

os.makedirs(os.path.dirname(path), exist_ok=True)
response = requests.get(url)
with open(path, 'w') as fd:
    fd.write(response.text)

print(f"File saved to {path}")

File saved to data/oni.ascii.txt


In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

oni = pd.read_csv(path, sep=r'\s+')
oni.head()

,SEAS,YR,TOTAL,ANOM
0,DJF,1950,24.72,-1.53
1,JFM,1950,25.17,-1.34
2,FMA,1950,25.75,-1.16
3,MAM,1950,26.12,-1.18
4,AMJ,1950,26.32,-1.07


In [5]:
oni_copy = oni.copy()
oni_copy['SEAS_YR'] = oni_copy['SEAS'] + ' ' + oni_copy['YR'].astype(str)
# oni_copy.drop(columns=['SEAS', 'YR'], inplace=True)
oni_copy.set_index('YR', inplace=True)
oni_copy['ANOM'] = oni_copy['ANOM'].round(1)
oni_copy.head()

,SEAS,TOTAL,ANOM,SEAS_YR
YR,,,,
1950,DJF,24.72,-1.5,DJF 1950
1950,JFM,25.17,-1.3,JFM 1950
1950,FMA,25.75,-1.2,FMA 1950
1950,MAM,26.12,-1.2,MAM 1950
1950,AMJ,26.32,-1.1,AMJ 1950


In [6]:
oni_copy.loc[oni_copy['SEAS_YR'] == 'ASO 1968']

,SEAS,TOTAL,ANOM,SEAS_YR
YR,,,,
1968,ASO,26.82,0.4,ASO 1968


In [7]:
i = 0
warm_periods = []
cold_periods = []

while i < len(oni_copy):
    val = oni_copy.iloc[i]['ANOM']
    
    if val >= 0.5:
        streak = 0
        j = i
        while j < len(oni_copy) and oni_copy.iloc[j]['ANOM'] >= 0.5:
            streak += 1
            j += 1
        if streak >= 5:
            start_date = oni_copy.iloc[i]['SEAS_YR']
            end_date = oni_copy.iloc[j - 1]['SEAS_YR']
            warm_periods.append([start_date, end_date])
            i = j
            continue

    elif val <= -0.5:
        streak = 0
        j = i
        while j < len(oni_copy) and oni_copy.iloc[j]['ANOM'] <= -0.5:
            streak += 1
            j += 1
        if streak >= 5:
            start_date = oni_copy.iloc[i]['SEAS_YR']
            end_date = oni_copy.iloc[j - 1]['SEAS_YR']
            cold_periods.append([start_date, end_date])
            i = j
            continue

    i += 1

In [8]:
import plotly.graph_objects as go

In [12]:
def plot_oni(oni, start=oni['YR'].min(), end=oni['YR'].max(), warm_periods=warm_periods, cold_periods=cold_periods):
    '''
    Plot the Oceanic Niño Index (ONI) with given start and end years.

    Parameters:
        oni: DataFrame containing the ONI data.
        start: Start year for the plot (default is the minimum year in the data).
        end: End year for the plot (default is the maximum year in the data).
    '''

    color = []
    oni_subset = oni.loc[start:end]

    for _, row in oni_subset.iterrows():
        if row['ANOM'] > 0:
            color.append('red')  # E41D3D
        else:
            color.append('blue')  # 0076D6

    fig = go.Bar(x=oni_subset['SEAS_YR'], y=oni_subset['ANOM'], marker_color=color)
    figure = go.Figure(data=fig)
    figure.add_hline(y=0, line_color='black', line_width=0.5)
    figure.add_hline(y=0.5, line_color='red', line_width=1.5)
    figure.add_hline(y=-0.5, line_color='blue', line_width=1.5)

    for start_period, end_period in warm_periods:
        figure.add_vrect(
            x0=start_period, x1=end_period,
            fillcolor="red", opacity=0.2, line_width=0,
            layer="below", annotation_text="", annotation_position="top left",
        )
    
    for start_period, end_period in cold_periods:
        figure.add_vrect(
            x0=start_period, x1=end_period,
            fillcolor="blue", opacity=0.2, line_width=0,
            layer="below", annotation_text="", annotation_position="top left"
        )

    figure.update_layout(
        title_text='Oceanic Niño Index (ONI)',
        template='none',
    )
    figure.update_layout(title_text='Oceanic Niño Index (ONI)', template='none')
    figure.show()

    # return figure

Warm and cold phases are defined as a minimum of five consecutive 3-month running averages of SST anomalies in the Niño 3.4 region surpassing a threshold of +/- 0.5°C.

In [10]:
#change values of start and end to plot different years
start = oni['YR'].min()
end = oni['YR'].max()

In [13]:
# figure = plot_oni(oni_copy, start, end, warm_periods, cold_periods)
plot_oni(oni_copy, start, end, warm_periods, cold_periods)